# 1. Ensemble of Random Forests with Resampling

In this notebook we use an enseble of random forest to predict the customer satisfaction. One of the more poignant characteristics of the data is the imbalance between the target classes, only ~4.5% of the observations are labeled as unsatisfied. To deal with this issue we undersample the majority class. The procedure is as follows (for details see train_forests.py):

- Split the training data according to the target variable: happy and unhappy customers.

- Randomly pick a specified fraction of the unhappy data.

- Merge the previous observations with a equally sized random sample of happy observations.

This gives us a balanced data set on which to train our random forests.

Remark: The metric that we use for evaluation is the area under the ROC curve.

In [27]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [29]:
# We import custom utility functions for data processing and random forest training
from process_data import process, create_submission, drop_columns
from train_forests import trainForests, mean_ensemble

In [30]:
#data = pd.read_csv('data/train_saldo.csv')
#data = pd.read_csv('data/train_extended_saldo.csv')
data = pd.read_csv('data/train.csv')
process(data)

In [31]:
train, test = train_test_split(data, test_size = 0.2, random_state = 42)

In [32]:
X_train, Y_train = train.ix[:,:-1], train['TARGET']

In [33]:
X_test, Y_test = test.ix[:,:-1], test['TARGET']

In [34]:
data.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,SumZeros,NumAssets,TARGET
0,0,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.576564,315,4,0
1,0,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.805234,289,9,0
2,0,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.117417,300,6,0
3,0,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.066763,270,12,0
4,0,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.672584,279,11,0


## 1.2 Single Forest Classifier

To have a better understanding of the problem, we examine the performance of a single random forest classifier.

In [35]:
rf = RandomForestClassifier(n_estimators = 150)
rf.fit(X_train, Y_train)
rf_bal = RandomForestClassifier(n_estimators = 150, class_weight='balanced')
rf_bal.fit(X_train, Y_train)
# custom undersambpling (see train_forests.py)
rf_custom = trainForests(train, n_trees = 150)[0]

In [36]:
# rf
Y_prob = pd.DataFrame(rf.predict_proba(X_test))[1]
Y_prob_train = pd.DataFrame(rf.predict_proba(X_train))[1]
print("rf --> Test score = {}, Train score = {}".format(roc_auc_score(Y_test, Y_prob), roc_auc_score(Y_train, Y_prob_train)))
# rf_bal
Y_prob = pd.DataFrame(rf_bal.predict_proba(X_test))[1]
Y_prob_train = pd.DataFrame(rf_bal.predict_proba(X_train))[1]
print("rf_bal --> Test score = {}, Train score = {}".format(roc_auc_score(Y_test, Y_prob), roc_auc_score(Y_train, Y_prob_train)))
# rf_custom
Y_prob = pd.DataFrame(rf_custom.predict_proba(X_test))[1]
Y_prob_train = pd.DataFrame(rf_custom.predict_proba(X_train))[1]
print("rf_custom --> Test score = {}, Train score = {}".format(roc_auc_score(Y_test, Y_prob), roc_auc_score(Y_train, Y_prob_train)))

rf --> Test score = 0.7628327185552672, Train score = 0.9975213935333158
rf_bal --> Test score = 0.7553627220686608, Train score = 0.9912921279519079
rf_custom --> Test score = 0.8040550522725947, Train score = 0.9400009333039534


From this we see that the built-in parameter to deal with imbalanced data sets, 'class_weight', is not really useful. However, our custom resampling really makes a difference.

## 1.3 Feature Importance

It is worthwile to try and understand what are the most important features according to a random forest classifier. One may even consider to train the final model using only some of the top features.

In [37]:
def feature_importance(n_trees, data):
    rf = RandomForestClassifier(n_trees)
    rf.fit(data.ix[:,:-1], data.ix[:,-1])
    fimp = rf.feature_importances_
    results = {}
    for idx, name in enumerate(data.ix[:,:-1].columns):
        results[name] = fimp[idx]
    return results

In [38]:
# We can use an initial random forest to take a look at important features
if False: # Set to 'False' to skip this step as it takes some time.
    threshold = 0.01
    rank = feature_importance(300, data)
    print("The rank of the features is as follows:")
    flag = 1
    count = 0
    for a in sorted(rank.keys(), key=rank.get)[::-1]:
        aux = rank[a]
        count+= 1
        if aux < threshold and flag:
            flag = 0
            print('---'*5)
            print('There are {} features above {} '.format(count, threshold))
            print('---'*5)
        print(a, '-->', rank[a])

**Important Remark**: This backs up our data analysis. Notice how all the new features that we created are on the top of the importance rank.

## 1.4 Ensemble of Random Forests

In the final part of the model, we train N_forest-many random forest with n_trees-many trees each. Ultimately, the prediction of the probability of being unsatisfied (TARGET = 0) is made as the geometric mean of the individual predictions.

The basic parameters in the model are:
- N_forest: number of random forests to train.
- n_trees: number of trees for each forest.
- a: the fraction of the TARGET == 1 class to be used in the training of each forest.
- w: weight of the TARGET == 0 class in the training set.

### Cross Validation

We perform a grid-search to find the best parameters, we obtain a=0.25, w=1, N_forest=90, n_trees=200.

In [39]:
cv_data = pd.read_csv("cross-validation/cross-val.csv")

In [40]:
cv_data.describe()

,a,w,N_forest,n_trees,auc_roc
count,332.000000,332.000000,332.000000,332.000000,332.000000
mean,0.717206,1.406627,40.451807,232.078313,0.805536
std,0.554718,0.716878,26.442220,136.373686,0.031897
min,0.111111,1.000000,10.000000,50.000000,0.720001
25%,0.250000,1.000000,20.000000,100.000000,0.786379
50%,0.500000,1.000000,40.000000,205.000000,0.820972
75%,1.000000,2.000000,50.000000,350.000000,0.828311
max,5.000000,3.000000,180.000000,500.000000,0.835501


In [41]:
cv_data[cv_data['auc_roc'] > 0.835]

,a,w,N_forest,n_trees,auc_roc
0,0.25,1,30,200,0.835188
5,0.25,1,60,300,0.835154
6,0.25,1,60,400,0.835114
7,0.25,1,60,500,0.835370
8,0.25,1,90,200,0.835501
331,0.25,1,60,150,0.835243


We compromise with N_forest = 30 and n_trees = 200 for the sake of
computer performance.

### Model Training

In [ ]:
a = 0.25 # It can also be >1 to oversample the minority class (1 ie. unhappy)
w = 1  # Weight of the majority class in the final sample.
N_forest = 90  # Number of forests to train
n_trees = 200  # Number of trees in each forest

In [ ]:
rfs = trainForests(train, a, w, N_forest, n_trees)

In [ ]:
# mean_ensemble takes the predicted probabilities of each random forest and computes the geometric mean. See train_forests.py for more details
Y_tot = mean_ensemble(rfs, X_test)

In [45]:
temp = roc_auc_score(Y_test,Y_tot['geometric'])
rf_classifiers = [rfs, temp]
print(temp)

0.8352569907


In [46]:
pickle.dump(rf_classifiers, open("models/rf_classifier_param1.dat", "wb"))

For comparison, we print the score of the prediction on the training set.

In [47]:
Y_prob = mean_ensemble(rfs, X_train)

In [48]:
roc_auc_score(Y_train, Y_prob["geometric"])

0.8656689630768486

This indicates that some overfitting might be going on. We are going to try and fix that.

#### Help with overfitting: 'min_samples_leaf'

There is another parameter that we can use to prevent overfitting, 'min_samples_leaf'. This is the number of samples that we require each leaf to have in each decision tree of the random forest. The higher the less prone to over fitting. We run a token experiment to see how scores change.

In [49]:
for msl in range(1, 11):
    rf = RandomForestClassifier(n_estimators = 300, min_samples_leaf = msl)
    rf.fit(train.ix[:,:-1],train['TARGET'])
    temp = pd.DataFrame(rf.predict_proba(X_test))[1]
    print("msl={} --> {}".format(msl,roc_auc_score(Y_test, temp)))

msl=1 --> 0.7655553447544401
msl=2 --> 0.8193590251613391
msl=3 --> 0.8196266209379982
msl=4 --> 0.820202612100453
msl=5 --> 0.8199581530316028
msl=6 --> 0.8177373676679066
msl=7 --> 0.8187904264591842
msl=8 --> 0.8175645195312751
msl=9 --> 0.8156901640437729
msl=10 --> 0.8177760793302409


A good choice for the parameter is msl = 5, let's see whether it improves our model.

In [52]:
a = 0.25 # It can also be >1 to oversample the minority class (1 ie. unhappy)
w = 1  # Weight of the majority class in the final sample.
N_forest = 60  # Number of forests to train
n_trees = 300  # Number of trees in each forest

In [53]:
for msl in range(1,6):
    rfs = trainForests(train, a, w, N_forest, n_trees, msl)
    Y_tot = mean_ensemble(rfs, X_test)
    temp = roc_auc_score(Y_test,Y_tot['geometric'])
    rf_classifiers.append([rfs, temp])
    Y_prob = mean_ensemble(rfs, X_train)
    print("msl={} --> Test error = {}, Training error = {}".format(msl, temp, roc_auc_score(Y_train, Y_prob["geometric"])))

msl=1 --> Test error = 0.8352349261809229, Training error = 0.8694161891445628
msl=2 --> Test error = 0.8180729063621319, Training error = 0.8466065649341591
msl=3 --> Test error = 0.8121969726125712, Training error = 0.8369061537207224
msl=4 --> Test error = 0.8065117756249479, Training error = 0.8297232318854276
msl=5 --> Test error = 0.8042192100360492, Training error = 0.8259110773803449


In [54]:
for idx in range(Y_tot.shape[1]-2):
    printroc_auc_scorelassifier = {}'.format(idx),'-->',roc_auc_score(Y_test, Y_tot.ix[:,idx]))

# classifier = 0 --> 0.790036013132
# classifier = 1 --> 0.79141394516
# classifier = 2 --> 0.794230923982
# classifier = 3 --> 0.794427247412
# classifier = 4 --> 0.793491226504
# classifier = 5 --> 0.797241178961
# classifier = 6 --> 0.790925365608
# classifier = 7 --> 0.795520090055
# classifier = 8 --> 0.803636672878
# classifier = 9 --> 0.80513739875
# classifier = 10 --> 0.786201075597
# classifier = 11 --> 0.804799094937
# classifier = 12 --> 0.793091525769
# classifier = 13 --> 0.794867239878
# classifier = 14 --> 0.803189231522
# classifier = 15 --> 0.8082328081
# classifier = 16 --> 0.8053983921
# classifier = 17 --> 0.793660293764
# classifier = 18 --> 0.800020856896
# classifier = 19 --> 0.794469457796
# classifier = 20 --> 0.792651702596
# classifier = 21 --> 0.794602239927
# classifier = 22 --> 0.786603710744
# classifier = 23 --> 0.805113302715
# classifier = 24 --> 0.799529004346
# classifier = 25 --> 0.800931596718
# classifier = 26 --> 0.790401742409
# classifier = 27

It doesn't help, at all. We should stick with the default msl=1 value.

In [26]:
pickle.dump(rf_classifiers, open("models/rf_msl_saldo_classifier_param1.dat", "wb"))

## Meta Ensemble

Now we are going to use the random forest classifiers trained above to create a meta predictor.

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
X_train, Y_train = train.ix[:,:-1], train['TARGET']

In [ ]:
X_train.head()

In [ ]:
for rf in rfs:
    temp = rf.predict(train.ix[:,:-1])
    temp = pd.DataFrame(temp)
    X_train = pd.concat([X_train, temp], axis=1)

In [ ]:
train_meta = pd.concat([X_train, train['TARGET']], axis=1)

In [ ]:
train_meta.head()

In [ ]:
X_test, Y_test = test.ix[:,:-1], test['TARGET']

In [ ]:
for rf in rfs:
    temp = rf.predict(test.ix[:,:-1])
    temp = pd.DataFrame(temp)
    X_test = pd.concat([X_test, temp], axis=1)

In [ ]:
Y_test = test['TARGET']

In [ ]:
X_test.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# random regularization
cv = {}

In [ ]:
for _ in range(20):
    n_trees = np.random.randint(50,300)
    lg = RandomForestClassifier(n_estimators = n_trees)
    lg.fit(X_train, Y_train)
    cv[n_trees] = roc_auc_score(Y_test, lg.predict(X_test))
    print(cv[n_trees])

In [ ]:
for key in sorted(cv.keys(), key=cv.get)[::-1]:
    print(key,cv[key])

### Score analysis

In [ ]:
X_test, Y_test = test.ix[:,:-1], test.ix[:,-1]
n = 100
a = 0.25
w = 1
N_forest = 5
n_trees = 5

In [ ]:
scores = []
for _ in range(n):
    rfs = trainForests(train, a, w, N_forest,n_trees)
    Y_prob = mean_ensemble(rfs, X_test)
    scores.append(roc_auc_score(Y_test,Y_prob))
scores = pd.DataFrame(scores)

In [ ]:
scores.describe()

In [ ]:
plt.title("Distribution of scores")
plt.hist(scores)
plt.show()

In [ ]:
# If desired, transform probabilities into class labels.
def threshold(Y_prob, threshold = 0.5):
    result = []
    for y in Y_prob:
        if y <= threshold:
            result.append(0)
        else:
            result.append(1)
    return result

In [ ]:
# Evaluate class labels
Y_pred = threshold(Y_prob, threshold = 0.5)
_ = eval_classification(test['TARGET'],Y_pred, print_results = True)

In [ ]:
# Plot feature importance
def plot_features(forest):  
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    n=len(indices)
    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(n), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(n), indices)
    plt.xlim([-1, n])
    plt.show()

In [ ]:
train.head()

# Create Submission

In [ ]:
# Retrain forest on the whole 'train.csv' data
rfs = trainForests(data, a, w, N_forest, n_trees, msl)

In [ ]:
test = pd.read_csv('data/test.csv')
test_id = test.ix[:,'ID'].values
process(test)

In [ ]:
Y_prob = mean_ensemble(rfs,test)

In [ ]:
create_submission(test_id, Y_prob['geometric'])

## Ensemble RF and XGBOOST

In [ ]:
Y_boost = pd.read_csv('../Kaggle_Santander-master/simplexgbtest.csv')

In [ ]:
Y_boost.head()

In [ ]:
Y_rf = pd.read_csv('submissions/rforest_ensemble2.csv')

In [ ]:
Y_rf.head()

In [ ]:
Y_prob = pd.concat([Y_boost,Y_rf.ix[:,'TARGET']], axis=1, ignore_index=True)

In [ ]:
Y_prob.rename(columns ={0:'ID', 1:'xgb', 2: 'rfe' }, inplace = True)

In [ ]:
# geometric mean ensemble
l = 2 #number of predictors to ensemble
temp = Y_prob.ix[:,1:].product(axis=1)
temp = temp.apply(lambda x: np.power(x, 1./l))
Y_prob['geometric'] = temp

In [ ]:
# arithmetic mean ensemble
l = 2 #number of predictors to ensemble
temp = Y_prob[['xgb', 'rfe']].mean(axis=1)
temp = temp.apply(lambda x: np.power(x, 1./l))
Y_prob['arithmetic'] = temp

In [ ]:
# difference column
temp = Y_prob['xgb'] - Y_prob['rfe']
Y_prob['xgb - rfe'] = temp

In [ ]:
# difference column
temp = Y_prob['geometric'] - Y_prob['arithmetic']
Y_prob['geo - ari'] = temp

In [ ]:
Y_prob.head()

In [ ]:
plt.title('Differences between XGB and RFE')
plt.hist(Y_prob['xgb - rfe'])
plt.show()

In [ ]:
plt.title('Differences between ensembles')
plt.hist(Y_prob['geo - ari'])
plt.show()

In [ ]:
create_submission(test_id, Y_prob['arithmetic'])